In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [2]:
predictors = ["cyl", "disp", "hp", "drat", "wt", "qsec"]
response = ["mpg"]

mtcars = pd.read_csv("mtcars.csv", usecols=response+predictors)
# mtcars = pd.concat([mtcars, pd.DataFrame(np.ones(shape=[32,1]))], axis=1)
mtcars.head()

,mpg,cyl,disp,hp,drat,wt,qsec
0,21.0,6,160.0,110,3.90,2.620,16.46
1,21.0,6,160.0,110,3.90,2.875,17.02
2,22.8,4,108.0,93,3.85,2.320,18.61
3,21.4,6,258.0,110,3.08,3.215,19.44
4,18.7,8,360.0,175,3.15,3.440,17.02


In [3]:
y = mtcars[response[0]]
X = mtcars.drop(["mpg"],axis=1)

In [4]:
X.shape

(32, 6)

In [5]:
reg = LinearRegression().fit(X, y)
coefficients = reg.coef_
intercepts = reg.intercept_
coefficients = np.hstack([coefficients, intercepts])
print(coefficients.shape)
print(intercepts.shape)

(7,)
()


In [6]:
coefficients

array([-8.18560235e-01,  1.32048951e-02, -1.79299325e-02,  1.32040573e+00,
       -4.19083238e+00,  4.01461166e-01,  2.63073590e+01])

In [7]:
intercepts

26.30735899381266

In [8]:
predictors.append("Y")

x_p = pd.concat([X, pd.DataFrame(np.ones(shape=[32,1]))], axis=1)
x_p.shape



(32, 7)

In [9]:
w_est = np.matmul(x_p, coefficients.T)
w_est

array([22.31414056, 21.47029655, 25.62378627, 21.22829659, 17.9505849 ,
       19.74604065, 15.75618159, 23.39138555, 24.35735289, 19.51002275,
       19.75089945, 14.15578146, 15.6609567 , 15.61199955, 11.40114204,
       10.36237375, 10.30351147, 26.87284082, 30.2768389 , 28.67755248,
       25.46815985, 16.93378362, 17.79272238, 15.00659058, 16.70110903,
       27.75854001, 26.57546067, 27.68471515, 17.7686536 , 19.56688034,
       13.3013209 , 23.92007897])

In [10]:
epsilon = w_est-y
N = len(y)
r = 7
dof = N-r-1
print(dof)
sq_s = np.matmul(epsilon.T, epsilon)/dof
t = 2.064
sq_tau_beta = np.linalg.inv(np.matmul(x_p.T, x_p))
sq_tau_beta.shape

24


(7, 7)

In [11]:
for ind, coe in enumerate(coefficients):
    interval = t*(sq_s*sq_tau_beta[ind][ind])**0.5
    print(predictors[ind], coe-interval, coe+interval)


cyl -2.528167227652163 0.8910467581248226
disp -0.012151202253758576 0.038560992458893543
hp -0.05059284498063068 0.014732980052926413
drat -1.7962008195594685 4.437012286373726
wt -6.840691027718206 -1.5409737267443346
qsec -0.686755034519867 1.4896773668878371
Y -4.511500171708757 57.12621815933407


In [12]:
# Backward Elimination
# The higher the error, the more we keep

In [13]:
def error(predictions, truth):
    return np.sum((truth-predictions)**2)

In [14]:
# cyl zero
co1 = coefficients.copy()
co1[0] = 0
est = np.matmul(x_p, co1.T)
error(est, y)

1050.6109110306768

In [15]:
# disp zero
co2 = coefficients.copy()
co2[1] = 0
est = np.matmul(x_p, co2.T)
error(est, y)

543.5368325246873

In [16]:
# hp zero
co3 = coefficients.copy()
co3[2] = 0
est = np.matmul(x_p, co3.T)
error(est, y)

431.68257403589155

In [17]:
# drat zero
co4 = coefficients.copy()
co4[3] = 0
est = np.matmul(x_p, co4.T)
error(est, y)

900.6002668241803

In [18]:
# wt zero
co5 = coefficients.copy()
co5[4] = 0
est = np.matmul(x_p, co5.T)
error(est, y)

6502.009741084888

In [19]:
# qsec zero
co6 = coefficients.copy()
co6[5] = 0
est = np.matmul(x_p, co6.T)
error(est, y)

1822.4880119182908

In [20]:
# intercept zero
co7 = coefficients.copy()
co7[6] = 0
est = np.matmul(x_p, co7.T)
error(est, y)

22309.945206461685

In [21]:
error(w_est, y)

163.47681512294227

In [22]:
backward_X = X.drop(["cyl", "disp", "hp", "drat"], axis=1)
backward_reg = LinearRegression().fit(backward_X, y)
backward_coefficients = backward_reg.coef_
backward_intercepts = backward_reg.intercept_
print(backward_coefficients, backward_intercepts)

[-5.04798198  0.92919798] 19.746222596481193


In [23]:
backward_estimator = np.matmul(backward_X, backward_coefficients.T) + backward_intercepts
error(backward_estimator, y)

195.46363160465626

In [24]:
# Order of importance: intercept, wt, qsec, cyl, drat, disp, hp
# Chosen to keep intercept, wt, qsec

In [25]:
# Forward Selection
# The lower the error, the more we keep

In [26]:
# cyl
co1 = np.zeros_like(coefficients)
co1[0] = coefficients[0]
est = np.matmul(x_p, co1.T)
error(est, y)

20976.31226215392

In [27]:
# disp
co2 = np.zeros_like(coefficients)
co2[1] = coefficients[1]
est = np.matmul(x_p, co2.T)
error(est, y)

11023.295856266632

In [28]:
# hp
co3 = np.zeros_like(coefficients)
co3[2] = coefficients[2]
est = np.matmul(x_p, co3.T)
error(est, y)

17335.750785849385

In [29]:
# drat
co4 = np.zeros_like(coefficients)
co4[3] = coefficients[3]
est = np.matmul(x_p, co4.T)
error(est, y)

8493.570655906999

In [30]:
# wt
co5 = np.zeros_like(coefficients)
co5[4] = coefficients[4]
est = np.matmul(x_p, co5.T)
error(est, y)

36387.7506594581

In [31]:
# qsec
co6 = np.zeros_like(coefficients)
co6[5] = coefficients[5]
est = np.matmul(x_p, co6.T)
error(est, y)

6375.58305153613

In [32]:
# intercept
co7 = np.zeros_like(coefficients)
co7[6] = coefficients[6]
est = np.matmul(x_p, co7.T)
error(est, y)

2362.7761970944284

In [33]:
# Order of importance = intercept, qsec, drat, disp, hp, cyl, wt
# Chosen to keep intercept, qsec, drat,

In [34]:
forward_X = X.drop(["disp", "hp", "cyl", "wt"], axis=1)
forward_reg = LinearRegression().fit(forward_X, y)
forward_coefficients = forward_reg.coef_
forward_intercepts =forward_reg.intercept_
print(forward_coefficients, forward_intercepts)

[7.30859187 1.21267514] -27.839917908689465


In [35]:
forward_estimator = np.matmul(forward_X, forward_coefficients.T) + forward_intercepts
error(forward_estimator, y)

459.20753751345114